In [1]:
import nltk
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import random
import spacy
from spacy.en import English
nlp = English()

In [2]:
def form_question(start, grouping, i):
    x = grouping[:i]+grouping[i+1:]
    x1 = [x[1] for x in x]
    x2 = " ".join(x1)
    return start+" "+x2+"?"

In [3]:
def squestion(sentence):
    check = nlp(unicode("Maybe "+sentence))
    check_tokens = [t.ent_type_ for t in check]
    ent = check_tokens[1]
    if ent=='':
        sentenceq = sentence[0].lower()+sentence[1:]
    subj = []
    mverb = []
    chunks = []
    for i in xrange(len(check[1:])):
    #for token in check[1:]:
        token = check[i+1]
        if token.dep_=='nsubj':
            subj=[i, token.orth_]
        if token.ent_type_=='':
            if token.pos_=='VERB' and token.dep_=='ROOT' and check[i].dep_!='aux':
                chunks.append(['VERB-aux', 'did'])
                chunks.append([token.pos_, token.lemma_])
                mverb=['did', token.orth_]
            elif token.pos_=='VERB' and token.dep_=='ROOT' and check[i].dep_=='aux':
                chunks.append([token.pos_, token.orth_])
                mverb=[check[i].orth_, token.orth_]
                
            #elif token.pos_=='VERB' and token.dep_=='aux':
                #chunks.append([token.pos_+'-aux', token.orth_, ])
            else:
                chunks.append([token.pos_, token.orth_])
        else:
            chunks.append([token.ent_type_, token.orth_])
    
    
    grouping = [chunks[0]]
    for item in chunks[1:]:
        #print (item[0], grouping[-1][0])
        if item[0]==grouping[-1][0]:
            grouping[-1][1]+=' '+item[1]
        else:
            grouping.append(item)
    quest_begin = {'PERSON':'Who', 'GPE':'Where', 'DATE':'When', 'NOUN':'What'}
    
    #for key in quest_begin.keys():
    #for item in grouping:
    qlist = []
    for i in xrange(len(grouping)):
        item = grouping[i]
        if item[0] in quest_begin.keys():
            start = quest_begin[item[0]]
            qlist.append(form_question(start, grouping, i))
    
    return qlist

In [6]:
squestion("Sergiy Savarin is going to deploy my solution tomorrow in Lviv")

[u'Who is going to deploy my solution tomorrow in Lviv?',
 u'What Sergiy Savarin is going to deploy my tomorrow in Lviv?',
 u'When Sergiy Savarin is going to deploy my solution in Lviv?',
 u'Where Sergiy Savarin is going to deploy my solution tomorrow in?']